In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
pd.set_option('display.max_columns',None) # To display all columns
pd.set_option('display.max_rows',None) # To display all columns
import feather
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import glob
import warnings
warnings.filterwarnings('ignore')
from functools import reduce

In [2]:
K1 = pd.read_feather('KT1.ftr')
#KT3 = pd.read_feather('KT3.ftr')

In [3]:
questions_df = pd.read_csv('questions.csv')
questions_df.head()

,question_id,bundle_id,explanation_id,correct_answer,part,tags,deployed_at
0,q1,b1,e1,b,1,1;2;179;181,1558093217098
1,q2,b2,e2,a,1,15;2;182,1558093219720
2,q3,b3,e3,b,1,14;2;179;183,1558093222784
3,q4,b4,e4,b,1,9;2;179;184,1558093225357
4,q5,b5,e5,c,1,8;2;179;181,1558093228439


In [4]:
K1.head()

,timestamp,solving_id,question_id,user_answer,elapsed_time,user_iD
0,1565096190868,1,q5012,b,38000,1
1,1565096221062,2,q4706,c,24000,1
2,1565096293432,3,q4366,b,68000,1
3,1565096339668,4,q4829,a,42000,1
4,1565096401774,5,q6528,b,59000,1


In [5]:
questions_df.bundle_id = questions_df.bundle_id.str.slice(start=1)
questions_df.explanation_id = questions_df.explanation_id.str.slice(start=1)
questions_df.deployed_at = pd.to_datetime(questions_df.deployed_at, unit='ms')
dfq = questions_df[['question_id', 'correct_answer', 'bundle_id', 'explanation_id' , 'part', 'tags', 'deployed_at']]
dfq.head()

,question_id,correct_answer,bundle_id,explanation_id,part,tags,deployed_at
0,q1,b,1,1,1,1;2;179;181,2019-05-17 11:40:17.098
1,q2,a,2,2,1,15;2;182,2019-05-17 11:40:19.720
2,q3,b,3,3,1,14;2;179;183,2019-05-17 11:40:22.784
3,q4,b,4,4,1,9;2;179;184,2019-05-17 11:40:25.357
4,q5,c,5,5,1,8;2;179;181,2019-05-17 11:40:28.439


In [7]:
# subset df for notnull question_id rows only (only work with rows that have questions)
df2 = K1[K1['question_id'].notnull()]

In [8]:
# left join student dataframe and answer key dataframe on the 'question_id' column 
dfm = pd.merge(df2, dfq,  how='left', left_on=['question_id'], right_on = ['question_id'])
dfm.dropna(axis = 0, how = 'all', inplace = True)

In [10]:
A = dfm['correct_answer']
B = dfm['user_answer']

In [11]:
# create a 'correct' column
dfm['correct'] = np.nan

# loop - if user answers == correct answer, then dfq['correct'][_]= 1
length_df = dfm['user_iD'].count()
count = 0
for i in range(length_df):
    if B[count] == A[count]:
        dfm['correct'][count] = 1
    if B[count] != A[count]:
        dfm['correct'][count] = 0
    count+=1

KeyboardInterrupt: 

In [ ]:
# inspect new merged dataframe
dfm.head()

In [ ]:
df_total_questions_attempted = dfm2.groupby(['user_iD'], as_index=False)['question_id'].count()

df_total_questions_attempted = df_total_questions_attempted.rename(columns={'question_id': 'Total Questions Attempted'})

df_total_questions_attempted.head()
                                                                   

In [ ]:
df_correct = dfm2[dfm2['user_correct']==1].groupby(['user_iD'], as_index=False)['question_id'].count()

df_correct = df_correct.rename(columns={'question_id': 'Corrected'})

df_correct.head()

In [ ]:
df_incorrect = dfm2[dfm2['user_correct']==0].groupby(['user_iD'], as_index=False)['question_id'].count()

df_incorrect = df_incorrect.rename(columns={'question_id': 'InCorrected'})

df_incorrect.head()

In [ ]:
df = pd.concat([df_total_questions_attempted, df_correct, df_incorrect], axis=1).fillna(0)
df.head()

In [ ]:
dfs = [df_total_questions_attempted, df_correct, df_incorrect]
df_final = reduce(lambda left,right: pd.merge(left,right,on='user_iD'), dfs)

In [ ]:
df_final.head()

In [ ]:
df_final['Percentage'] = (df_final['Corrected'] / df_final['Total Questions Attempted']) *100
df_final.head()

In [ ]:
A = df_final.loc[df_final['Percentage'] >= 50]
A.head()

In [ ]:
A.plot(x="user_iD", y=["Percentage"], kind="bar",figsize=(15,10) , rot=0)